In [1]:
import pandas as pd
import talib
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter


In [24]:
WINDOW_MA = 10
WINDOW_RSI = 10
SLIDING_WINDOW = 5
EPSILON_TOLERANCE = 0.15
TRAINING_PCT = 0.8

Lectura de archivos

In [3]:
tweets_df = pd.read_csv("../data/tweets_with_sentiment_cryptobert.csv")
price_df = pd.read_csv("../data/datos_mercado_binance.csv")

In [4]:
tweets_df.head(5)

,Username,Text,Created At,Retweets,Likes,Sentiment
0,Peter McCormack 🏴‍☠️🇬🇧🇮🇪,Happy New Year to you all.\n\nI hope you all h...,2023-01-01 01:49:27+00:00,8.0,450.0,1
1,Mohamed A. El-Erian,The notion of an exciting football game means ...,2023-01-01 01:55:27+00:00,3.0,419.0,1
2,Jimmy Song (송재준),"10 years ago, I:\n\n* Bought more Bitcoin\n* S...",2023-01-01 06:48:00+00:00,21.0,531.0,1
3,Jameson Lopp,Bitcoin average DAILY value change during:\n20...,2023-01-01 12:50:57+00:00,35.0,231.0,0
4,Jameson Lopp,Bitcoin average DAILY value change during:\n20...,2023-01-01 12:50:57+00:00,35.0,231.0,0


In [5]:
price_df.head(5)

,simbolo,fecha,precio_apertura,precio_maximo,precio_minimo,precio_cierre,volumen
0,BTCUSDT,2023-01-01 00:00:00,16541.77,16628.00,16499.01,16616.75,96925.41374
1,BTCUSDT,2023-01-02 00:00:00,16617.17,16799.23,16548.70,16672.87,121888.57191
2,BTCUSDT,2023-01-03 00:00:00,16672.78,16778.40,16605.28,16675.18,159541.53733
3,BTCUSDT,2023-01-04 00:00:00,16675.65,16991.87,16652.66,16850.36,220362.18862
4,BTCUSDT,2023-01-05 00:00:00,16850.36,16879.82,16753.00,16831.85,163473.56641


In [6]:

tweets_df['fecha'] = pd.to_datetime(tweets_df['Created At'], errors='coerce')

# Asigna dia, mes y año a la nueva columna fecha quitando la hora
tweets_df['fecha'] = tweets_df['fecha'].dt.date
price_df['fecha'] = pd.to_datetime(price_df['fecha'], errors='coerce')
tweets_df.head(5)


,Username,Text,Created At,Retweets,Likes,Sentiment,fecha
0,Peter McCormack 🏴‍☠️🇬🇧🇮🇪,Happy New Year to you all.\n\nI hope you all h...,2023-01-01 01:49:27+00:00,8.0,450.0,1,2023-01-01
1,Mohamed A. El-Erian,The notion of an exciting football game means ...,2023-01-01 01:55:27+00:00,3.0,419.0,1,2023-01-01
2,Jimmy Song (송재준),"10 years ago, I:\n\n* Bought more Bitcoin\n* S...",2023-01-01 06:48:00+00:00,21.0,531.0,1,2023-01-01
3,Jameson Lopp,Bitcoin average DAILY value change during:\n20...,2023-01-01 12:50:57+00:00,35.0,231.0,0,2023-01-01
4,Jameson Lopp,Bitcoin average DAILY value change during:\n20...,2023-01-01 12:50:57+00:00,35.0,231.0,0,2023-01-01


In [7]:
# We count by kind of feeling
count_sentiment = tweets_df.groupby(['fecha', 'Sentiment']).size().unstack(fill_value=0)

# We recalculate totals and average
count_sentiment['numero_noticias'] = count_sentiment.sum(axis=1)
count_sentiment['promedio_sentimiento'] = tweets_df.groupby('fecha')['Sentiment'].mean()

Transforma las columnas date en formato datetime.date

In [8]:
price_df['fecha'] = pd.to_datetime(price_df['fecha']).dt.date 
count_sentiment = count_sentiment.reset_index()  # si 'fecha' está en el índice
count_sentiment['fecha'] = pd.to_datetime(count_sentiment['fecha']).dt.date
count_sentiment.rename(
    columns={
        -1:'negative_sentiment_news',
        0:'neutral_sentiment_news',
        1:'positive_sentiment_news'
    },
    inplace=True
)


In [9]:
count_sentiment.head(5)


Sentiment,fecha,negative_sentiment_news,neutral_sentiment_news,positive_sentiment_news,numero_noticias,promedio_sentimiento
0,2023-01-01,0,10,5,15,0.333333
1,2023-01-02,0,2,0,2,0.000000
2,2023-01-03,0,5,3,8,0.375000
3,2023-01-04,2,4,5,11,0.272727
4,2023-01-05,1,6,6,13,0.384615


Realiza la fusión de los dos DataFrames (price_df y conteo_sentimientos) usando la columna fecha como clave



In [10]:
# We join by date
df_final = price_df.merge(count_sentiment, on='fecha', how='left')

In [11]:
df_final.head(5)

,simbolo,fecha,precio_apertura,precio_maximo,precio_minimo,precio_cierre,volumen,negative_sentiment_news,neutral_sentiment_news,positive_sentiment_news,numero_noticias,promedio_sentimiento
0,BTCUSDT,2023-01-01,16541.77,16628.00,16499.01,16616.75,96925.41374,0.0,10.0,5.0,15.0,0.333333
1,BTCUSDT,2023-01-02,16617.17,16799.23,16548.70,16672.87,121888.57191,0.0,2.0,0.0,2.0,0.000000
2,BTCUSDT,2023-01-03,16672.78,16778.40,16605.28,16675.18,159541.53733,0.0,5.0,3.0,8.0,0.375000
3,BTCUSDT,2023-01-04,16675.65,16991.87,16652.66,16850.36,220362.18862,2.0,4.0,5.0,11.0,0.272727
4,BTCUSDT,2023-01-05,16850.36,16879.82,16753.00,16831.85,163473.56641,1.0,6.0,6.0,13.0,0.384615


CALCULO MEDIA MOVIL CON DERIVADA Y TENDENCIA

In [12]:

# We added trend column
df_final['media_movil_10d'] = df_final['precio_apertura'].rolling(WINDOW_MA).mean()

# PROFE TIPS: la derivada hecha con la funcion diff no es la mejor opcion
# revisar otras funciones de derivadas

# LISTO CORREGIDO
df_final['mm10_smooth'] = savgol_filter(df_final['media_movil_10d'], WINDOW_MA+1, polyorder=2)
df_final['derivada_mm10'] = np.gradient(df_final['mm10_smooth'])



In [13]:
epsilon = df_final['derivada_mm10'].std() * EPSILON_TOLERANCE
condition_list = [
    df_final['derivada_mm10'].isna() | ((df_final['derivada_mm10'] > -epsilon) & (df_final['derivada_mm10'] < epsilon)),
    df_final['derivada_mm10'] > 0,
    df_final['derivada_mm10'] < 0,
]
returns_values_list = [0, 1, -1]
df_final['tendencia_categorizada'] = np.select(condition_list, returns_values_list, 0)

📈 Indicador RSI (Relative Strength Index)

📊 RSI toma valores entre 0 y 100:

- (<30) podría indicar sobreventa.
- (>70) podría indicar sobrecompra.

In [14]:
# PROFE TIPS: trata de no usar funciones creadas por ti para los indicadores
# LISTO CORREGIDO

df_final['RSI_10'] = talib.RSI(df_final['precio_apertura'], WINDOW_RSI)

🔁 Indicador MACD (Moving Average Convergence Divergence)


In [15]:
# EMA rápida y lenta
ema_12 = df_final['precio_apertura'].ewm(span=12, adjust=False).mean()
ema_26 = df_final['precio_apertura'].ewm(span=26, adjust=False).mean()

# Línea MACD y señal
df_final['MACD'] = ema_12 - ema_26
df_final['MACD_signal'] = df_final['MACD'].ewm(span=9, adjust=False).mean()

In [16]:
df_final

,simbolo,fecha,precio_apertura,precio_maximo,precio_minimo,precio_cierre,volumen,negative_sentiment_news,neutral_sentiment_news,positive_sentiment_news,numero_noticias,promedio_sentimiento,media_movil_10d,mm10_smooth,derivada_mm10,tendencia_categorizada,RSI_10,MACD,MACD_signal
0,BTCUSDT,2023-01-01,16541.77,16628.00,16499.01,16616.75,96925.41374,0.0,10.0,5.0,15.0,0.333333,NaN,NaN,NaN,0,NaN,0.000000,0.000000
1,BTCUSDT,2023-01-02,16617.17,16799.23,16548.70,16672.87,121888.57191,0.0,2.0,0.0,2.0,0.000000,NaN,NaN,NaN,0,NaN,6.014815,1.202963
2,BTCUSDT,2023-01-03,16672.78,16778.40,16605.28,16675.18,159541.53733,0.0,5.0,3.0,8.0,0.375000,NaN,NaN,NaN,0,NaN,15.094857,3.981342
3,BTCUSDT,2023-01-04,16675.65,16991.87,16652.66,16850.36,220362.18862,2.0,4.0,5.0,11.0,0.272727,NaN,NaN,NaN,0,NaN,22.265775,7.638228
4,BTCUSDT,2023-01-05,16850.36,16879.82,16753.00,16831.85,163473.56641,1.0,6.0,6.0,13.0,0.384615,NaN,NaN,NaN,0,NaN,41.567254,14.424034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
877,BTCUSDT,2025-05-27,109434.78,110718.00,107516.57,108938.17,21276.65635,1.0,10.0,8.0,19.0,0.368421,107686.404,107646.127074,339.913637,1,66.138775,3769.126180,3868.602046
878,BTCUSDT,2025-05-28,108938.17,109284.70,106769.43,107781.78,15633.78829,2.0,24.0,13.0,39.0,0.282051,108267.556,107948.184134,264.200483,1,63.750214,3639.414671,3822.764571
879,BTCUSDT,2025-05-29,107781.78,108891.91,105322.86,105589.75,19834.70116,7.0,25.0,13.0,45.0,0.133333,108400.307,108174.528041,188.487329,1,58.302505,3404.066398,3739.024936
880,BTCUSDT,2025-05-30,105589.75,106313.12,103621.00,103985.48,23706.49799,0.0,16.0,6.0,22.0,0.272727,108401.909,108325.158793,112.774176,1,49.409607,3006.021155,3592.424180


Creación df con indicadores finales

In [17]:
# Selecting final columns
df_csv = df_final[['fecha',
                    'precio_apertura',
                    'media_movil_10d', 
                    'RSI_10', 
                    'MACD_signal', 
                    'tendencia_categorizada', 
                    'numero_noticias', 
                    'negative_sentiment_news', 
                    'neutral_sentiment_news', 
                    'positive_sentiment_news', 
                    'promedio_sentimiento']]

# We rename columns
df_csv.columns = ['date', 
                  'open',
                  'MA',
                  'RSI',
                  'MACD', 
                  'trend', 
                  'issue_news', 
                  'negative_sentiment_news', 
                  'neutral_sentiment_news', 
                  'positive_sentiment_news', 
                  'average_news']



In [18]:
df_final.head(5)

,simbolo,fecha,precio_apertura,precio_maximo,precio_minimo,precio_cierre,volumen,negative_sentiment_news,neutral_sentiment_news,positive_sentiment_news,numero_noticias,promedio_sentimiento,media_movil_10d,mm10_smooth,derivada_mm10,tendencia_categorizada,RSI_10,MACD,MACD_signal
0,BTCUSDT,2023-01-01,16541.77,16628.00,16499.01,16616.75,96925.41374,0.0,10.0,5.0,15.0,0.333333,NaN,NaN,NaN,0,NaN,0.000000,0.000000
1,BTCUSDT,2023-01-02,16617.17,16799.23,16548.70,16672.87,121888.57191,0.0,2.0,0.0,2.0,0.000000,NaN,NaN,NaN,0,NaN,6.014815,1.202963
2,BTCUSDT,2023-01-03,16672.78,16778.40,16605.28,16675.18,159541.53733,0.0,5.0,3.0,8.0,0.375000,NaN,NaN,NaN,0,NaN,15.094857,3.981342
3,BTCUSDT,2023-01-04,16675.65,16991.87,16652.66,16850.36,220362.18862,2.0,4.0,5.0,11.0,0.272727,NaN,NaN,NaN,0,NaN,22.265775,7.638228
4,BTCUSDT,2023-01-05,16850.36,16879.82,16753.00,16831.85,163473.56641,1.0,6.0,6.0,13.0,0.384615,NaN,NaN,NaN,0,NaN,41.567254,14.424034


In [19]:
df_csv.head(5)

,date,open,MA,RSI,MACD,trend,issue_news,negative_sentiment_news,neutral_sentiment_news,positive_sentiment_news,average_news
0,2023-01-01,16541.77,NaN,NaN,0.000000,0,15.0,0.0,10.0,5.0,0.333333
1,2023-01-02,16617.17,NaN,NaN,1.202963,0,2.0,0.0,2.0,0.0,0.000000
2,2023-01-03,16672.78,NaN,NaN,3.981342,0,8.0,0.0,5.0,3.0,0.375000
3,2023-01-04,16675.65,NaN,NaN,7.638228,0,11.0,2.0,4.0,5.0,0.272727
4,2023-01-05,16850.36,NaN,NaN,14.424034,0,13.0,1.0,6.0,6.0,0.384615


Creación ventana deslizante

In [20]:
# Features
colums_windows = ['open','MA','RSI','MACD', 'trend', 'issue_news', 
                    'negative_sentiment_news', 'neutral_sentiment_news', 'positive_sentiment_news', 'average_news']

# Lista para almacenar las wind
wind = []

# Recorremos desde el día 1 hasta len (cantidad de filas del df) - ventana 
for i in range(len(df_csv) - SLIDING_WINDOW):
    # Extraemos las filas de la ventana anterior (días pasados)
    segment = df_csv[colums_windows].iloc[i:i+SLIDING_WINDOW]
    
    # Aplanamos los datos de la ventana en una sola fila
    row_wind = segment.values.flatten()

    date = df_csv['date'].iloc[i-1 + SLIDING_WINDOW]
    #Precio objetivo: el siguiente día después de la ventana
    target_trend = df_csv['trend'].iloc[i + SLIDING_WINDOW]

    # Se crea la fila con target_date, target_trend y la ventana
    wind.append([date] + row_wind.tolist() + [target_trend])
    
columns_name = ['date']
# Generacion de columnas "open_d1, RS1_d1, open_d2, etc" de la ventana deslizante


x = (np.arange(SLIDING_WINDOW) * -1)[::-1]
for i, day in enumerate(x) :
    for col in colums_windows:
        columns_name.append(f'{col}_d{day}')
# Se crea la columna de target
columns_name.append('target_trend')



Exportar csv

In [ ]:
# DataFrame final
df_sliding_window = pd.DataFrame(wind, columns=columns_name)

In [22]:
df_sliding_window.shape

(877, 52)

In [ ]:
# Exportar a CSV
df_sliding_window.to_csv('../data/final_data.csv', index=False)

In [25]:
training_set, test_set = df_sliding_window[: int(len(df_sliding_window) * TRAINING_PCT)], df_sliding_window[int(len(df_sliding_window) * TRAINING_PCT):]

In [29]:
training_set.to_csv('../data/training_set.csv', index=False)
test_set.to_csv('../data/test_set.csv', index=False)